In [99]:
import pandas as pd
import numpy as np
import os
import pickle

def summarize_metrics(path, show_train=True, show_test=True, show_scalar=True, runs=11):
    results = pd.read_pickle(path)

    train_metric_sums = {}
    test_metric_sums = {}
    scalar_sums = {
        'training_time_sec': 0.0,
        'gpu_mem_MB': 0.0,
        'cpu_mem_MB': 0.0,
        'param_count': 0.0
    }

    count = 0
    for run in range(runs):
        if run not in results:
            continue
        entry = results[run]
        count += 1

        for key in scalar_sums:
            scalar_sums[key] += entry.get(key, 0.0)

        for k, v in entry.get('train_metrics', {}).items():
            train_metric_sums[k] = train_metric_sums.get(k, 0.0) + float(v)

        for k, v in entry.get('test_metrics', {}).items():
            test_metric_sums[k] = test_metric_sums.get(k, 0.0) + float(v)

    # 평균 계산
    scalar_avg = {f"avg_{k}": round(v / count, 2) for k, v in scalar_sums.items()} if show_scalar else {}
    train_avg = {f"train_{k}": round(v / count, 2) for k, v in train_metric_sums.items()} if show_train else {}
    test_avg = {f"test_{k}": round(v / count, 2) for k, v in test_metric_sums.items()} if show_test else {}

    # 통합 및 DataFrame 변환
    all_metrics = {**scalar_avg, **train_avg, **test_avg}
    df = pd.DataFrame([all_metrics])

    return df

dataset_order = [
    "basic", "gaussian", "uniform", "outlier", "edge", "BA", "ER", "grid", "tree",
    "education", "election", "income", "unemployment",
    "PTBR", "chameleon", "crocodile", "squirrel", "Anaheim", "ChicagoSketch"
]

def get_dataset_index(filename):
    for i, name in enumerate(dataset_order):
        if name in filename:
            return i
    return len(dataset_order)  # 못 찾은 건 맨 뒤로

def summarize_model_resources(path, runs=11):
    results = pd.read_pickle(path)

    models = ['gnn', 'conf_gnn']
    scalar_keys = ['training_time_sec', 'gpu_mem_MB', 'cpu_mem_MB', 'param_count']

    # 모델별 누적 딕셔너리 초기화
    model_sums = {model: {k: 0.0 for k in scalar_keys} for model in models}
    model_counts = {model: 0 for model in models}

    for run in range(runs):
        if run not in results:
            continue
        for model in models:
            entry = results[run].get(model, {})
            if not entry:
                continue
            model_counts[model] += 1
            for key in scalar_keys:
                model_sums[model][key] += entry.get(key, 0.0)

    # 평균 계산
    model_avgs = {}
    for model in models:
        if model_counts[model] == 0:
            continue
        avg_metrics = {
            f"{model}_{k}": round(v / model_counts[model], 4)
            for k, v in model_sums[model].items()
        }
        model_avgs.update(avg_metrics)

    df = pd.DataFrame([model_avgs])
    return df

## Alone Model

### SQR

In [53]:
train, test, scalar = False, False, True

folder_path = "/home/sypark/GQNN/pred/SQR"
pkl_files = [f for f in os.listdir(folder_path) if f.endswith('.pkl')]
sorted_pkl_files = sorted(pkl_files, key=get_dataset_index)

for i in range(len(pkl_files)):
    print('SQR', sorted_pkl_files[i], 'Results...')
    path = os.path.join(folder_path, sorted_pkl_files[i])
    print(summarize_metrics(path=path, show_train=train, show_test=test, show_scalar=scalar).T) 
    print('-' * 30)

SQR basic_SQR.pkl Results...
                              0
avg_training_time_sec     1.800
avg_gpu_mem_MB           65.789
avg_cpu_mem_MB         1231.970
avg_param_count        9153.000
------------------------------
SQR gaussian_SQR.pkl Results...
                              0
avg_training_time_sec     1.887
avg_gpu_mem_MB           65.789
avg_cpu_mem_MB         1230.541
avg_param_count        9153.000
------------------------------
SQR uniform_SQR.pkl Results...
                              0
avg_training_time_sec     1.889
avg_gpu_mem_MB           65.789
avg_cpu_mem_MB         1232.142
avg_param_count        9153.000
------------------------------
SQR outlier_SQR.pkl Results...
                              0
avg_training_time_sec     1.978
avg_gpu_mem_MB           65.789
avg_cpu_mem_MB         1232.871
avg_param_count        9153.000
------------------------------
SQR edge_SQR.pkl Results...
                              0
avg_training_time_sec     1.937
avg_gpu_mem_MB       

### RQR

In [54]:
train, test, scalar = False, False, True

folder_path = "/home/sypark/GQNN/pred/RQR"
pkl_files = [f for f in os.listdir(folder_path) if f.endswith('.pkl')]
sorted_pkl_files = sorted(pkl_files, key=get_dataset_index)

for i in range(len(pkl_files)):
    print('RQR', sorted_pkl_files[i], 'Results...')
    path = os.path.join(folder_path, sorted_pkl_files[i])
    print(summarize_metrics(path=path, show_train=train, show_test=test, show_scalar=scalar).T) 
    print('-' * 30)

RQR basic_RQR.pkl Results...
                              0
avg_training_time_sec     2.068
avg_gpu_mem_MB           85.999
avg_cpu_mem_MB         1294.955
avg_param_count        9090.000
------------------------------
RQR gaussian_RQR.pkl Results...
                              0
avg_training_time_sec     1.913
avg_gpu_mem_MB           85.999
avg_cpu_mem_MB         1295.988
avg_param_count        9090.000
------------------------------
RQR uniform_RQR.pkl Results...
                              0
avg_training_time_sec     1.900
avg_gpu_mem_MB           85.999
avg_cpu_mem_MB         1295.865
avg_param_count        9090.000
------------------------------
RQR outlier_RQR.pkl Results...
                              0
avg_training_time_sec     2.041
avg_gpu_mem_MB           85.999
avg_cpu_mem_MB         1295.324
avg_param_count        9090.000
------------------------------
RQR edge_RQR.pkl Results...
                              0
avg_training_time_sec     1.912
avg_gpu_mem_MB       

### BNN

In [55]:
train, test, scalar = False, False, True

folder_path = "/home/sypark/GQNN/pred/BNN"
pkl_files = [f for f in os.listdir(folder_path) if f.endswith('.pkl')]
sorted_pkl_files = sorted(pkl_files, key=get_dataset_index)

for i in range(len(pkl_files)):
    print('BNN', sorted_pkl_files[i], 'Results...')
    path = os.path.join(folder_path, sorted_pkl_files[i])
    print(summarize_metrics(path=path, show_train=train, show_test=test, show_scalar=scalar).T) 
    print('-' * 30)

BNN basic_BNN.pkl Results...
                              0
avg_training_time_sec     1.693
avg_gpu_mem_MB           65.769
avg_cpu_mem_MB         1263.186
avg_param_count        9090.000
------------------------------
BNN gaussian_BNN.pkl Results...
                              0
avg_training_time_sec     1.560
avg_gpu_mem_MB           65.769
avg_cpu_mem_MB         1263.401
avg_param_count        9090.000
------------------------------
BNN uniform_BNN.pkl Results...
                              0
avg_training_time_sec     1.713
avg_gpu_mem_MB           65.769
avg_cpu_mem_MB         1262.642
avg_param_count        9090.000
------------------------------
BNN outlier_BNN.pkl Results...
                              0
avg_training_time_sec     1.432
avg_gpu_mem_MB           65.769
avg_cpu_mem_MB         1263.419
avg_param_count        9090.000
------------------------------
BNN edge_BNN.pkl Results...
                              0
avg_training_time_sec     1.453
avg_gpu_mem_MB       

### MC

In [56]:
train, test, scalar = False, False, True

folder_path = "/home/sypark/GQNN/pred/MC"
pkl_files = [f for f in os.listdir(folder_path) if f.endswith('.pkl')]
sorted_pkl_files = sorted(pkl_files, key=get_dataset_index)

for i in range(len(pkl_files)):
    print('MC', sorted_pkl_files[i], 'Results...')
    path = os.path.join(folder_path, sorted_pkl_files[i])
    print(summarize_metrics(path=path, show_train=train, show_test=test, show_scalar=scalar).T) 
    print('-' * 30)

MC basic_MC.pkl Results...
                              0
avg_training_time_sec     1.428
avg_gpu_mem_MB           66.009
avg_cpu_mem_MB         1237.782
avg_param_count        9025.000
------------------------------
MC gaussian_MC.pkl Results...
                              0
avg_training_time_sec     1.461
avg_gpu_mem_MB           66.009
avg_cpu_mem_MB         1237.172
avg_param_count        9025.000
------------------------------
MC uniform_MC.pkl Results...
                              0
avg_training_time_sec     1.460
avg_gpu_mem_MB           66.009
avg_cpu_mem_MB         1236.890
avg_param_count        9025.000
------------------------------
MC outlier_MC.pkl Results...
                              0
avg_training_time_sec     1.465
avg_gpu_mem_MB           66.009
avg_cpu_mem_MB         1236.305
avg_param_count        9025.000
------------------------------
MC edge_MC.pkl Results...
                              0
avg_training_time_sec     1.441
avg_gpu_mem_MB           66.009

### CP

In [57]:
train, test, scalar = False, False, True

folder_path = "/home/sypark/GQNN/pred/CP"
pkl_files = [f for f in os.listdir(folder_path) if f.endswith('.pkl')]
sorted_pkl_files = sorted(pkl_files, key=get_dataset_index)

for i in range(len(pkl_files)):
    print('CP', sorted_pkl_files[i], 'Results...')
    path = os.path.join(folder_path, sorted_pkl_files[i])
    print(summarize_metrics(path=path, show_train=train, show_test=test, show_scalar=scalar).T) 
    print('-' * 30)

CP basic_CP.pkl Results...
                              0
avg_training_time_sec     1.372
avg_gpu_mem_MB           65.678
avg_cpu_mem_MB         1229.489
avg_param_count        9025.000
------------------------------
CP gaussian_CP.pkl Results...
                              0
avg_training_time_sec     1.549
avg_gpu_mem_MB           65.678
avg_cpu_mem_MB         1229.168
avg_param_count        9025.000
------------------------------
CP uniform_CP.pkl Results...
                              0
avg_training_time_sec     1.597
avg_gpu_mem_MB           65.678
avg_cpu_mem_MB         1230.165
avg_param_count        9025.000
------------------------------
CP outlier_CP.pkl Results...
                              0
avg_training_time_sec     1.705
avg_gpu_mem_MB           65.678
avg_cpu_mem_MB         1229.977
avg_param_count        9025.000
------------------------------
CP edge_CP.pkl Results...
                              0
avg_training_time_sec     1.728
avg_gpu_mem_MB           65.678

### CF-GNN

In [13]:
train, test, scalar = True, True, False

folder_path = "/home/sypark/GQNN/pred/CF-GNN (opt)"
pkl_files = [f for f in os.listdir(folder_path) if f.endswith('.pkl')]
sorted_pkl_files = sorted(pkl_files, key=get_dataset_index)

for i in range(len(pkl_files)):
    print('CF-GNN', sorted_pkl_files[i], 'Results...')
    path = os.path.join(folder_path, sorted_pkl_files[i])
    print(summarize_metrics(path=path, show_train=train, show_test=test, show_scalar=scalar).T) 
    print('-' * 30)

CF-GNN optimal_education_GraphSAGE_conftr_calib_holdout_confgnn.pkl Results...
                       0
train_PCIP        0.9043
train_MPIW        3.1736
train_NMPIW       0.5244
train_MCT         0.0250
train_CWC         1.0270
train_MPE         0.7883
train_Sharpness  12.4961
train_WS          3.2223
test_PCIP         0.9042
test_MPIW         3.0964
test_NMPIW        0.4944
test_MCT          0.0547
test_CWC          0.9753
test_MPE          0.7613
test_Sharpness   11.8171
test_WS           3.1382
------------------------------
CF-GNN optimal_election_GraphSAGE_conftr_calib_holdout_confgnn.pkl Results...
                      0
train_PCIP       0.9010
train_MPIW       0.8963
train_NMPIW      0.5155
train_MCT        0.0079
train_CWC        1.0284
train_MPE        0.2197
train_Sharpness  0.8135
train_WS         0.9093
test_PCIP        0.9110
test_MPIW        0.9439
test_NMPIW       0.5408
test_MCT         0.0161
test_CWC         1.0289
test_MPE         0.2249
test_Sharpness   0.9008
tes

### GQNN

In [100]:
train, test, scalar = True, True, False

folder_path = "/home/sypark/GQNN/pred/GQNN"
pkl_files = [f for f in os.listdir(folder_path) if f.endswith('.pkl')]
sorted_pkl_files = sorted(pkl_files, key=get_dataset_index)

for i in range(len(pkl_files)):
    print('GQNN', sorted_pkl_files[i], 'Results...')
    path = os.path.join(folder_path, sorted_pkl_files[i])
    print(summarize_metrics(path=path, show_train=train, show_test=test, show_scalar=scalar).T) 
    print('-' * 30)

GQNN basic_GQNN_lf(0.05).pkl Results...
                    0
train_PCIP       0.88
train_MPIW       0.17
train_NMPIW      0.17
train_MCT        0.01
train_CWC        0.39
train_MPE        0.05
train_Sharpness  0.04
train_WS         0.17
test_PCIP        0.80
test_MPIW        0.18
test_NMPIW       0.18
test_MCT         0.02
test_CWC         0.67
test_MPE         0.05
test_Sharpness   0.04
test_WS          0.18
------------------------------
GQNN gaussian_GQNN_lf(0.05).pkl Results...
                    0
train_PCIP       0.90
train_MPIW       0.37
train_NMPIW      0.37
train_MCT        0.00
train_CWC        0.73
train_MPE        0.10
train_Sharpness  0.14
train_WS         0.37
test_PCIP        0.85
test_MPIW        0.39
test_NMPIW       0.45
test_MCT         0.02
test_CWC         1.16
test_MPE         0.11
test_Sharpness   0.16
test_WS          0.40
------------------------------
GQNN uniform_GQNN_lf(0.05).pkl Results...
                    0
train_PCIP       0.89
train_MPIW       0.32

## Ablation

In [ ]:
def summarize_ablation_results(result_dir):
    rows = []

    for fname in os.listdir(result_dir):
        if not fname.endswith('.pkl'):
            continue

        config_name = fname.replace('.pkl', '')
        with open(os.path.join(result_dir, fname), 'rb') as f:
            results = pickle.load(f)

        for run_id, result in results.items():
            row = {'config': config_name, 'run': run_id}

            # 기본 성능 지표
            for k, v in result['test_metrics'].items():
                row[f'test_{k}'] = v

            # loss 항목
            if 'loss_terms' in result:
                for k, v in result['loss_terms'].items():
                    row[f'loss_{k}'] = v

            # 기타
            row['train_time_sec'] = result.get('training_time_sec', None)
            row['gpu_mem_MB'] = result.get('gpu_mem_MB', None)
            row['param_count'] = result.get('param_count', None)

            rows.append(row)

    df = pd.DataFrame(rows)
    return df

dataset = 'tree'
result_dir = f"./ablation/{dataset}/"  # 실험 결과 저장 디렉토리
df_summary = summarize_ablation_results(result_dir)
df_summary.to_csv(f"./ablation/{dataset}_ablation_summary.csv", index=False)
print("Ablation summary saved to CSV.")

configs_to_include_arch = [
    # Architecture ablation
    "dual_output(1)_fixed_margin(None)_use_sample_loss(1)_use_coverage_loss(1)_use_width_loss(1)",
    "dual_output(1)_fixed_margin(0.05)_use_sample_loss(1)_use_coverage_loss(1)_use_width_loss(1)",
    "dual_output(1)_fixed_margin(0.10)_use_sample_loss(1)_use_coverage_loss(1)_use_width_loss(1)",
    "dual_output(0)_fixed_margin(None)_use_sample_loss(1)_use_coverage_loss(1)_use_width_loss(1)",
]

configs_to_include_loss = [
    # Loss ablation
    "dual_output(1)_fixed_margin(None)_use_sample_loss(1)_use_coverage_loss(1)_use_width_loss(1)",
    "dual_output(1)_fixed_margin(None)_use_sample_loss(1)_use_coverage_loss(1)_use_width_loss(0)",
    "dual_output(1)_fixed_margin(None)_use_sample_loss(1)_use_coverage_loss(0)_use_width_loss(1)",
    "dual_output(1)_fixed_margin(None)_use_sample_loss(0)_use_coverage_loss(1)_use_width_loss(1)",
    "dual_output(1)_fixed_margin(None)_use_sample_loss(0)_use_coverage_loss(0)_use_width_loss(1)",
    "dual_output(1)_fixed_margin(None)_use_sample_loss(1)_use_coverage_loss(0)_use_width_loss(0)",
]

Ablation summary saved to CSV.


In [102]:
pd.read_csv(f"./ablation/{dataset}_ablation_summary.csv").groupby('config').mean().round(2).loc[configs_to_include_arch].iloc[:, :3]

,run,test_PCIP,test_MPIW
config,,,
dual_output(1)_fixed_margin(None)_use_sample_loss(1)_use_coverage_loss(1)_use_width_loss(1),2.0,0.84,0.15
dual_output(1)_fixed_margin(0.05)_use_sample_loss(1)_use_coverage_loss(1)_use_width_loss(1),2.0,0.73,0.10
dual_output(1)_fixed_margin(0.10)_use_sample_loss(1)_use_coverage_loss(1)_use_width_loss(1),2.0,0.88,0.20
dual_output(0)_fixed_margin(None)_use_sample_loss(1)_use_coverage_loss(1)_use_width_loss(1),2.0,0.79,0.16


In [111]:
pd.read_csv(f"./ablation/{dataset}_ablation_summary.csv").groupby('config').mean().round(2).loc[configs_to_include_loss].iloc[:, :3]

,run,test_PCIP,test_MPIW
config,,,
dual_output(1)_fixed_margin(None)_use_sample_loss(1)_use_coverage_loss(1)_use_width_loss(1),1.0,1.00,0.50
dual_output(1)_fixed_margin(None)_use_sample_loss(1)_use_coverage_loss(1)_use_width_loss(0),1.0,1.00,1.09
dual_output(1)_fixed_margin(None)_use_sample_loss(1)_use_coverage_loss(0)_use_width_loss(1),1.0,0.99,0.50
dual_output(1)_fixed_margin(None)_use_sample_loss(0)_use_coverage_loss(1)_use_width_loss(1),1.0,0.02,0.13
dual_output(1)_fixed_margin(None)_use_sample_loss(0)_use_coverage_loss(0)_use_width_loss(1),1.0,0.02,0.13
dual_output(1)_fixed_margin(None)_use_sample_loss(1)_use_coverage_loss(0)_use_width_loss(0),1.0,1.00,1.10


In [ ]:
train, test, scalar = True, True, False

folder_path = "/home/sypark/GQNN/ablation/chameleon"
pkl_files = [f for f in os.listdir(folder_path) if f.endswith('.pkl')]
pkl_files.sort()

for i in range(len(pkl_files)):
    print('Education', pkl_files[i], 'Results...')
    path = os.path.join(folder_path, pkl_files[i])
    # print(summarize_metrics(path=path, show_train=train, show_test=test, show_scalar=scalar).T) 
    df = summarize_metrics(path=path, show_train=train, show_test=test, show_scalar=scalar).T
    print(df.round(4))
    print('-' * 30)

### education

In [ ]:
train, test, scalar = False, True, False

folder_path = "/home/sypark/GQNN/ablation/education"
pkl_files = [f for f in os.listdir(folder_path) if f.endswith('.pkl')]
pkl_files.sort()

for i in range(len(pkl_files)):
    print('Education', pkl_files[i], 'Results...')
    path = os.path.join(folder_path, pkl_files[i])
    print(summarize_metrics(path=path, show_train=train, show_test=test, show_scalar=scalar).T) 
    print('-' * 30)

Education dual_output(0)_use_coverage_loss(0)_use_width_loss(0).pkl Results...
                      0
train_PCIP       0.9995
train_MPIW       0.9768
train_NMPIW      0.9768
train_MCT        0.0972
train_CWC        1.3378
train_MPE        0.2524
train_Sharpness  0.9546
train_WS         0.9768
test_PCIP        0.9920
test_MPIW        0.9764
test_NMPIW       0.7501
test_MCT         0.0898
test_CWC         1.0492
test_MPE         0.2544
test_Sharpness   0.9542
test_WS          0.9770
------------------------------
Education dual_output(0)_use_coverage_loss(0)_use_width_loss(1).pkl Results...
                      0
train_PCIP       0.8973
train_MPIW       0.3179
train_NMPIW      0.3179
train_MCT        0.0012
train_CWC        0.6445
train_MPE        0.0833
train_Sharpness  0.1119
train_WS         0.3221
test_PCIP        0.8997
test_MPIW        0.3197
test_NMPIW       0.2456
test_MCT         0.0014
test_CWC         0.4922
test_MPE         0.0836
test_Sharpness   0.1166
test_WS          0.

### PTBR

In [ ]:
train, test, scalar = True, True, False

folder_path = "/home/sypark/GQNN/ablation/PTBR"
pkl_files = [f for f in os.listdir(folder_path) if f.endswith('.pkl')]
pkl_files.sort()

for i in range(len(pkl_files)):
    print('Education', pkl_files[i], 'Results...')
    path = os.path.join(folder_path, pkl_files[i])
    print(summarize_metrics(path=path, show_train=train, show_test=test, show_scalar=scalar).T) 
    print('-' * 30)

Education dual_output(0)_use_coverage_loss(0)_use_width_loss(0).pkl Results...
                      0
train_PCIP       0.9992
train_MPIW       1.3476
train_NMPIW      1.3476
train_MCT        0.1338
train_CWC        1.8471
train_MPE        0.2193
train_Sharpness  1.9288
train_WS         1.3476
test_PCIP        0.9796
test_MPIW        1.0717
test_NMPIW       1.7575
test_MCT         0.0847
test_CWC         2.5560
test_MPE         0.1804
test_Sharpness   1.3200
test_WS          1.0729
------------------------------
Education dual_output(0)_use_coverage_loss(0)_use_width_loss(1).pkl Results...
                      0
train_PCIP       0.9467
train_MPIW       0.0802
train_NMPIW      0.0802
train_MCT        0.0044
train_CWC        0.1354
train_MPE        0.0206
train_Sharpness  0.0081
train_WS         0.0806
test_PCIP        0.5749
test_MPIW        0.0725
test_NMPIW       0.1188
test_MCT         0.0199
test_CWC         4.3903
test_MPE         0.0342
test_Sharpness   0.0067
test_WS          0.

### chameleon

In [ ]:
train, test, scalar = False, True, False

folder_path = "/home/sypark/GQNN/ablation/chameleon"
pkl_files = [f for f in os.listdir(folder_path) if f.endswith('.pkl')]
pkl_files.sort()

for i in range(len(pkl_files)):
    print('Education', pkl_files[i], 'Results...')
    path = os.path.join(folder_path, pkl_files[i])
    print(summarize_metrics(path=path, show_train=train, show_test=test, show_scalar=scalar).T) 
    print('-' * 30)

Education dual_output(0)_use_coverage_loss(0)_use_width_loss(0).pkl Results...
                      0
train_PCIP       0.9997
train_MPIW       0.4051
train_NMPIW      0.4051
train_MCT        0.0404
train_CWC        0.5546
train_MPE        0.1717
train_Sharpness  0.1701
train_WS         0.4051
test_PCIP        0.9824
test_MPIW        0.4020
test_NMPIW       0.6213
test_MCT         0.0335
test_CWC         0.8927
test_MPE         0.1705
test_Sharpness   0.1665
test_WS          0.4031
------------------------------
Education dual_output(0)_use_coverage_loss(0)_use_width_loss(1).pkl Results...
                      0
train_PCIP       0.9105
train_MPIW       0.0099
train_NMPIW      0.0099
train_MCT        0.0003
train_CWC        0.0192
train_MPE        0.0027
train_Sharpness  0.0003
train_WS         0.0100
test_PCIP        0.3873
test_MPIW        0.0090
test_NMPIW       0.0139
test_MCT         0.0045
test_CWC         2.5364
test_MPE         0.0218
test_Sharpness   0.0002
test_WS          0.

### anaheim

In [23]:
train, test, scalar = True, True, False

folder_path = "/home/sypark/GQNN/ablation/Anaheim"
pkl_files = [f for f in os.listdir(folder_path) if f.endswith('.pkl')]
pkl_files.sort()

for i in range(len(pkl_files)):
    print('Education', pkl_files[i], 'Results...')
    path = os.path.join(folder_path, pkl_files[i])
    print(summarize_metrics(path=path, show_train=train, show_test=test, show_scalar=scalar).T) 
    print('-' * 30)

Education dual_output(0)_use_coverage_loss(0)_use_width_loss(0).pkl Results...
                      0
train_PCIP       0.9995
train_MPIW       1.0857
train_NMPIW      1.0857
train_MCT        0.1080
train_CWC        1.4874
train_MPE        0.3380
train_Sharpness  1.1925
train_WS         1.0857
test_PCIP        1.0000
test_MPIW        0.9648
test_NMPIW       1.2836
test_MCT         0.0965
test_CWC         1.7559
test_MPE         0.2941
test_Sharpness   0.9580
test_WS          0.9648
------------------------------
Education dual_output(0)_use_coverage_loss(0)_use_width_loss(1).pkl Results...
                      0
train_PCIP       0.8809
train_MPIW       0.2651
train_NMPIW      0.2651
train_MCT        0.0051
train_CWC        0.5884
train_MPE        0.0768
train_Sharpness  0.0947
train_WS         0.2689
test_PCIP        0.8516
test_MPIW        0.2881
test_NMPIW       0.3832
test_MCT         0.0140
test_CWC         1.0119
test_MPE         0.0900
test_Sharpness   0.1063
test_WS          0.